# <p style="background-color:skyblue; font-family:newtimeroman; font-size:160%; text-align:center">Tabular Playground Series April</p>
1. [Exploratory Data Analysis 📊](https://www.kaggle.com/zhaodianwen/tps-april-1-eda/)
2. [Data Pre-processing 📝](https://www.kaggle.com/zhaodianwen/tps-april-2-data-pre-processing/)
3. [Model Development 🧰](https://www.kaggle.com/zhaodianwen/tps-april-3-model-development/)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):

    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
###To continue with previous EDA & Data Pre-processing

##loading data
train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

##deal with missing data
test['Embarked'].fillna((train['Embarked'].mode()), inplace=True)
train['Embarked'].fillna((train['Embarked'].mode()), inplace=True)

test['Fare'].fillna((train['Fare'].mean()), inplace=True)
train['Fare'].fillna((train['Fare'].mean()), inplace=True)
train['Fare'] = train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
test['Fare'] = test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

test['Age'].fillna((train['Age'].median()), inplace=True)
train['Age'].fillna((train['Age'].median()), inplace=True)

##combine the number of SibSp & Parch +1 to be FamilySize, a new feature synthesized:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1 
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1 

## Encodeing Categorical Columns:

## Dropping unimportant columns:
train.drop(['Name','Ticket','Cabin'], axis=1, inplace=True)
test.drop(['Name','Ticket','Cabin'], axis=1, inplace=True)

## Applying Label Encoder:
obj_cols = train.select_dtypes(include=['object']).columns.tolist()
for col in obj_cols:
    le = LabelEncoder()
    le.fit(train[col])
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

## Defining crutial features and the target:

In [ ]:
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','FamilySize']

target = train['Survived'].values

# Naive Models 👓

## Logistic Regression Model

In [ ]:
lr = LogisticRegression()
lr.fit(train[features], target)
print('Logistic Regression Model\'s ROC AUC score is:', roc_auc_score(target, lr.predict_proba(train[features])[:,1]))
print('Logistic Regression Model\'s accuracy score is:', accuracy_score(target, lr.predict(train[features])))

In [ ]:
submission['Survived'] = lr.predict(test[features])
submission.to_csv('submission-lr.csv', index=False)
submission.head()